In [2]:
import pandas as pd 
import numpy as np 

In [3]:
#importamos los datasets previamente transformados
#Primero ponemos la ruta
ruta_games = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\games.parquet"
ruta_items = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\users_items.parquet"
ruta_reviews = r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\PI1\PI MLOps - STEAM\Datos Estructurados\df_review_final.parquet"
#luego los extraemos
games = pd.read_parquet(ruta_games)
#items = pd.read_parquet(ruta_items)
reviews = pd.read_parquet(ruta_reviews)

In [4]:
games.head(5)

,App_Name,Id,Genres,Developer,Publisher,Price,Release_Date
0,Lost Summoner Kitty,761140,"[Action, Casual, Indie, Simulation, Strategy]",Kotoshiro,Kotoshiro,4.99,2018-01-04
1,Ironbound,643980,"[Free to Play, Indie, RPG, Strategy]",Secret Level SRL,"Making Fun, Inc.",0.00,2018-01-04
2,Real Pool 3D - Poolians,670290,"[Casual, Free to Play, Indie, Simulation, Sports]",Poolians.com,Poolians.com,0.00,2017-07-24
3,弹炸人2222,767400,"[Action, Adventure, Casual]",彼岸领域,彼岸领域,0.99,2017-12-07
5,Battle Royale Trainer,772540,"[Action, Adventure, Simulation]",Trickjump Games Ltd,Trickjump Games Ltd,3.99,2018-01-04


In [5]:
reviews.head(5)

,User_Id,Item_Id,Recommend,Analisis_Sentimiento
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,2


In [6]:
# Examinamos los datasets que nos serviran para hacer la funcion #4
# Usaremos los datasets games y reviews
# Arreglamos los datasets seleccionando las columnas que necesitamos
games_col = games[["App_Name", "Id","Developer"]]
games_col.rename(columns={"Id": "Item_Id"}, inplace=True)
games_col.head(5)

C:\Users\Cristian\AppData\Local\Temp\ipykernel_8588\1309929762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_col.rename(columns={"Id": "Item_Id"}, inplace=True)


,App_Name,Item_Id,Developer
0,Lost Summoner Kitty,761140,Kotoshiro
1,Ironbound,643980,Secret Level SRL
2,Real Pool 3D - Poolians,670290,Poolians.com
3,弹炸人2222,767400,彼岸领域
5,Battle Royale Trainer,772540,Trickjump Games Ltd


In [7]:
# Eliminamos datos nulos
reviews = reviews.dropna(subset=["Item_Id"])
games_col = games_col.dropna(subset=["Item_Id"])
# Convertimos a tipo de dato
reviews["Item_Id"] = reviews["Item_Id"].astype("int32")
funcion_5 = games_col.merge(reviews, on="Item_Id", how="inner")
funcion_5.head(4)

,App_Name,Item_Id,Developer,User_Id,Recommend,Analisis_Sentimiento
0,Carmageddon Max Pack,282010,Stainless Games Ltd,InstigatorAU,True,1
1,Half-Life,70,Valve,EizanAratoFujimaki,True,2
2,Half-Life,70,Valve,GamerFag,True,0
3,Half-Life,70,Valve,76561198020928326,True,2


In [8]:
# Organizamos el orden de las columnas
funcion_5 = funcion_5[["App_Name", "Item_Id", "User_Id", "Developer", "Recommend", "Analisis_Sentimiento"]]
funcion_5.head(5)

,App_Name,Item_Id,User_Id,Developer,Recommend,Analisis_Sentimiento
0,Carmageddon Max Pack,282010,InstigatorAU,Stainless Games Ltd,True,1
1,Half-Life,70,EizanAratoFujimaki,Valve,True,2
2,Half-Life,70,GamerFag,Valve,True,0
3,Half-Life,70,76561198020928326,Valve,True,2
4,Half-Life,70,Bluegills,Valve,True,2


In [9]:
# Eliminamos las columnas que no son utiles para el analisis
funcion_5.drop(columns=["Recommend", "App_Name", "Item_Id", "User_Id"], inplace=True)

In [10]:
funcion_5.head(5)

,Developer,Analisis_Sentimiento
0,Stainless Games Ltd,1
1,Valve,2
2,Valve,0
3,Valve,2
4,Valve,2


In [11]:
funcion_5 = funcion_5[funcion_5["Analisis_Sentimiento"] != 1]
funcion_5.head(5)

,Developer,Analisis_Sentimiento
1,Valve,2
2,Valve,0
3,Valve,2
4,Valve,2
5,Valve,2


In [12]:
funcion_5["Developer"].unique()

array(['Valve', 'Outerlight Ltd.', 'GlyphX Games', ..., 'Neversoft',
       'Malfador Machinations', 'Strategy First'], dtype=object)

In [13]:
# def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
# Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}
def developer_reviews_analysis( desarrollador : str):
    '''
    Esta funcion devuelve un diccionario en el cual la clave es el desarrollador y su valor es una lista 
    en donde se detallan la cantidad reseñas de caracter positivo y negativo

    Args:
        desarrollador (str): Nombre del desarrollador.

    Returns:
        dict: Un diccionario que contiene información sobre el desarrollador y la cantidad de reseñas positivas y negativas acerca de él.
            - Desarrolador (int): Nombre de la empresa desarrolladora.
            - 'Negative' (int): Cantidad de reseñas negativas.
            - 'Positive' (float): Cantidad de reseñas negativas.

    '''
    filtered_dev = funcion_5[funcion_5["Developer"] == desarrollador]

    if filtered_dev.empty:
        return f"No se encontraron datos para el desarrollador {desarrollador}"
    
    Positive_filter = filtered_dev[filtered_dev["Analisis_Sentimiento"] == 2]
    Negative_filter = filtered_dev[filtered_dev["Analisis_Sentimiento"] == 0]

    reviews = {
        "Negative": Negative_filter.shape[0],  
        "Positive": Positive_filter.shape[0]
    }

    resultado = {desarrollador: reviews}

    return resultado

    



In [14]:
developer_reviews_analysis('Valve')

{'Valve': {'Negative': 957, 'Positive': 5882}}

In [15]:
funcion_5.to_parquet("Dataset_Funcion_5.parquet", engine="pyarrow")